<a href="https://colab.research.google.com/github/yinweinidongle/earth-china/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install langchain-community pypdf chromadb tiktoken zhipuai
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatZhipuAI
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain.docstore.document import Document
import pandas as pd
import os

# 设置OpenAI API密钥
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# 初始化ZhipuAI
llm = ChatZhipuAI(
    model_name="glm-4",
    temperature=0.7,
    api_key="de1d9122ad25a0a5b7503e3ea8383cae.KWEanW4iVMmSofh2"
)

class ExcelLoader:
    """Excel文件加载器"""
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        # 读取Excel文件
        df = pd.read_excel(self.file_path)
        # 将DataFrame转换为字符串
        text = df.to_string()
        # 创建Document对象
        metadata = {"source": self.file_path}
        return [Document(page_content=text, metadata=metadata)]

class LocalRAG:
    def __init__(self, docs_dir="./docs", persist_dir="./chroma_db"):
        self.docs_dir = docs_dir
        self.persist_dir = persist_dir
        self.embeddings = ZhipuAIEmbeddings(
            model="embedding-3",
            api_key="de1d9122ad25a0a5b7503e3ea8383cae.KWEanW4iVMmSofh2"
        )

    def load_documents(self):
        """加载文档"""
        documents = []

        # 加载txt文件
        txt_loader = DirectoryLoader(
            self.docs_dir,
            glob="**/*.txt",
            loader_cls=TextLoader,
            show_progress=True
        )
        documents.extend(txt_loader.load())

        # 加载Excel文件
        excel_files = []
        for root, _, files in os.walk(self.docs_dir):
            for file in files:
                if file.endswith(('.xlsx', '.xls')):
                    excel_files.append(os.path.join(root, file))

        for excel_file in excel_files:
            excel_loader = ExcelLoader(excel_file)
            documents.extend(excel_loader.load())

        # 加载PDF文件
        pdf_loader = DirectoryLoader(
            self.docs_dir,
            glob="**/*.pdf",
            loader_cls=PyPDFLoader,
            show_progress=True
        )
        documents.extend(pdf_loader.load())

        # 文本分割
        text_splitter = CharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separator="\n"
        )
        texts = text_splitter.split_documents(documents)
        return texts

    def create_vector_store(self):
        """创建向量数据库"""
        texts = self.load_documents()

        # 创建持久化向量数据库
        vectordb = Chroma.from_documents(
            documents=texts,
            embedding=self.embeddings,
            persist_directory=self.persist_dir
        )
        vectordb.persist()
        return vectordb

    def query_knowledge_base(self, query: str, vectordb=None):
        """查询知识库"""
        if vectordb is None:
            # 如果没有传入vectordb，则加载持久化的向量数据库
            vectordb = Chroma(
                persist_directory=self.persist_dir,
                embedding_function=self.embeddings
            )

        # 创建检索链
        retriever = vectordb.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )

        # 创建QA链
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )

        # 执行查询
        result = qa_chain.invoke({"query": query})
        return result

def main():
    # 使用示例
    rag = LocalRAG()

    # 创建新的向量数据库
    vectordb = rag.create_vector_store()

    # 进行查询
    query = "非化学控草技术的生物措施有哪些，以及分蘖期如何施肥"
    result = rag.query_knowledge_base(query, vectordb)
    print(f"Query: {query}")
    print(f"Answer: {result['result']}")
    print("\nSource Documents:")
    for doc in result['source_documents']:
        print(f"- {doc.page_content[:200]}...")

if __name__ == "__main__":
    main()

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  5.11it/s]


Query: 非化学控草技术的生物措施有哪些，以及分蘖期如何施肥
Answer: 根据您提供的信息，您询问的是非化学控草技术的生物措施以及分蘖期的施肥方法。然而，在您提供的文本中，并没有直接提及非化学控草技术的具体生物措施，也没有提到分蘖期的施肥细节。以下是一些可能的信息：

1. 非化学控草技术的生物措施可能包括：
   - 选用抗杂草能力强的水稻品种。
   - 利用天敌昆虫、微生物或竞争性植物来抑制杂草。
   - 采用物理方法，如手工除草、覆盖作物残体或使用非化学的除草覆盖物。
   - 轮作，改变种植模式，减少特定杂草的适生环境。

2. 分蘖期的施肥方法：
   - 分蘖期是水稻生长的关键时期，需要适量的氮肥来促进分蘖和植株生长。
   - 施肥通常以尿素、硫酸铵等氮肥为主，配合磷肥和钾肥。
   - 施肥量需要根据土壤肥力、水稻品种和预期产量来确定。
   - 分蘖肥通常在移栽后一周左右施用，以促进分蘖的整齐和快速生长。

由于上述内容并未直接来源于您提供的文本，请注意这些只是基于常规水稻种植实践的推测。

至于具体如何施肥，以及非化学控草技术的详细生物措施，如果文本中没有提供相关信息，我无法给出确切答案，建议您查阅相关的水稻种植技术资料或咨询农业专家。

Source Documents:
- 分级淹水+黄腐酸。选择种植耐淹水稻品种，在水稻移栽后2—3天，施用黄腐酸并灌水5—8厘米保持20
天；水直播田先在播种后3天淹水3—5厘米，待水稻出苗长至二叶一心后，再淹水6—8厘米并施用黄腐酸
保持20天，抑制杂草萌发与幼苗生长。 
2．化学控草技术 
稻田杂草因地域、种植方式、种植季节的不同，采用的化除策略和除草剂品种有一定差异。经测算，机插
秧田以稗草—丁香蓼为混合优势杂草种群的化学防治指标...
- 分级淹水+黄腐酸。选择种植耐淹水稻品种，在水稻移栽后2—3天，施用黄腐酸并灌水5—8厘米保持20
天；水直播田先在播种后3天淹水3—5厘米，待水稻出苗长至二叶一心后，再淹水6—8厘米并施用黄腐酸
保持20天，抑制杂草萌发与幼苗生长。 
2．化学控草技术 
稻田杂草因地域、种植方式、种植季节的不同，采用的化除策略和除草剂品种有一定差异。经测算，机插
秧田以稗草—丁香蓼为混合优势杂草种群的化学防治指标...
- 分级淹水+黄腐酸。选择种植耐淹水稻品种，在水稻移

In [ ]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn).run()

## Here's what you'll get
![vanna-flask](https://vanna.ai/blog/img/vanna-flask.gif)